In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Current Status,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,KL,Recovered,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,KL,Recovered,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(10060, 26)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
State code                                   0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Type of transmission                         0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(10060, 19)
Patient Number 	 819
State Patient Number 	 8405
Date Announced 	 819
Age Bracket 	 8857
Gender 	 8031
Detected City 	 8930
Detected District 	 2233
Detected State 	 820
State code 	 821
Current Status 	 820
Notes 	 2011
Contracted from which Patient (Suspected) 	 8710
Nationality 	 8840
Type of transmission 	 7204
Status Change Date 	 930
Source_1 	 1055
Source_2 	 7627
Source_3 	 9749
Backup Notes 	 9700


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(10060, 19)


Patient Number                                819
State Patient Number                         8405
Date Announced                                819
Age Bracket                                  8857
Gender                                       8031
Detected City                                8930
Detected District                            2233
Detected State                                820
State code                                    821
Current Status                                820
Notes                                        2011
Contracted from which Patient (Suspected)    8710
Nationality                                  8840
Type of transmission                         7204
Status Change Date                            930
Source_1                                     1055
Source_2                                     7627
Source_3                                     9749
Backup Notes                                 9700
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(9240, 19)


Patient Number                                  0
State Patient Number                         7585
Date Announced                                  0
Age Bracket                                  8038
Gender                                       7212
Detected City                                8110
Detected District                            1414
Detected State                                  0
State code                                      1
Current Status                                  1
Notes                                        1192
Contracted from which Patient (Suspected)    7890
Nationality                                  8021
Type of transmission                         6385
Status Change Date                            111
Source_1                                      236
Source_2                                     6808
Source_3                                     8930
Backup Notes                                 8880
dtype: int64

In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')

In [15]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
1297,1298,NaN,30/03/2020,NaN,M,Nizamuddin area,South Delhi,Delhi,DL,Hospitalized,Travelled to Delhi,E0,NaN,Local,30/03/2020,https://twitter.com/ANI/status/124464075167946...,NaN,NaN,NaN
4764,4765,NaN,06/04/2020,NaN,NaN,NaN,Kota,Rajasthan,RJ,Hospitalized,Details awaited,NaN,NaN,NaN,06/04/2020,https://twitter.com/PIBJaipur/status/124719774...,NaN,NaN,NaN
679,680,NaN,26/03/2020,12,M,NaN,Munger,Bihar,BR,Hospitalized,Relatives of P348,P348,India,Local,26/03/2020,https://twitter.com/PTI_News/status/1243073634...,https://economictimes.indiatimes.com/news/poli...,NaN,NaN
5313,5314,NaN,07/04/2020,NaN,NaN,NaN,NaN,Delhi,DL,Hospitalized,Details awaited,NaN,NaN,NaN,07/04/2020,Press Bulletin,NaN,NaN,NaN
3175,3176,NaN,04/04/2020,NaN,NaN,Navi Mumbai,Thane,Maharashtra,MH,Hospitalized,NaN,NaN,NaN,NaN,04/04/2020,NaN,https://twitter.com/ANI/status/124632346167436...,https://twitter.com/PTI_News/status/1246336675...,NaN


In [16]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'state_code', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'type_of_transmission',
       'status_change_date', 'source_1', 'source_2', 'source_3',
       'backup_notes', 'p_id'],
      dtype='object')

In [17]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [18]:
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,state_code,current_status,notes,suspected_contacted_patient,nationality,type_of_transmission,status_change_date,source_1,source_2,source_3,backup_notes
3718,3719,NaN,05/04/2020,26,M,NaN,Bhavnagar,Gujarat,GJ,Hospitalized,Local Transmission,NaN,India,Local,05/04/2020,https://gujcovid19.gujarat.gov.in/,https://gujcovid19.gujarat.gov.in/uploads/05-p...,NaN,NaN
6704,6705,NaN,09/04/2020,NaN,NaN,NaN,Other States*,Maharashtra,MH,Hospitalized,Details awaited,NaN,NaN,NaN,09/04/2020,https://twitter.com/rajeshtope11/status/124828...,NaN,NaN,NaN
4935,4936,TN-P622,07/04/2020,NaN,F,NaN,Thoothukkudi,Tamil Nadu,TN,Hospitalized,Details awaited,NaN,NaN,NaN,07/04/2020,https://twitter.com/ANI/status/124750529084656...,NaN,NaN,NaN
6184,6185,NaN,09/04/2020,NaN,NaN,NaN,Mumbai,Maharashtra,MH,Hospitalized,Details awaited,NaN,NaN,NaN,09/04/2020,https://twitter.com/PTI_News/status/1248126814...,NaN,NaN,NaN
4742,4743,NaN,06/04/2020,NaN,NaN,NaN,Indore,Madhya Pradesh,MP,Hospitalized,Details awaited,NaN,NaN,NaN,06/04/2020,https://twitter.com/JansamparkMP/status/124718...,NaN,NaN,NaN


In [19]:
p_df.to_csv('patients_data.csv', index=False)